# Synthetic Optical Flow from Fused Lidar


In [ ]:
import sys
sys.path.append('/opt/psegs')

import numpy as np

from psegs.exp.fused_lidar_flow import FusedLidarCloudTableBase

import IPython.display
import PIL.Image


## General Notebook Utilities
    
def imshow(x):
    IPython.display.display(PIL.Image.fromarray(x))

def show_html(x):
    from IPython.core.display import display, HTML
    display(HTML(x))

## SemanticKITTI

In [ ]:
from psegs.exp.semantic_kitti import SemanticKITTISDTable
class SemanticKITTIFusedWorldCloudTable(FusedLidarCloudTableBase):
    SRC_SD_TABLE = SemanticKITTISDTable

    # SemanticKITTI has no cuboids, so we skip this step.
    HAS_OBJ_CLOUDS = False
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        seg_rdd = cls.SRC_SD_TABLE.get_segment_datum_rdd(spark, segment_uri)
        
        # SemanticKITTI has no cuboids, so the Fuser algo simply concats the cloud points
        def iter_task_rows(iter_sds):
            from pyspark import Row
            from oarphpy.spark import RowAdapter
            for sd in iter_sds:
                if sd.point_cloud is not None:
                    pc = sd.point_cloud
                    task_id = "%s.%s" % (sd.uri.segment_id, pc.extra['semantic_kitti.scan_id'])
                    yield Row(
                        task_id=task_id,
                        point_clouds=[pc],
                        cuboids=[])
        
        task_rdd = seg_rdd.mapPartitions(iter_task_rows)
        import pyspark
        task_rdd = task_rdd.persist(pyspark.StorageLevel.DISK_ONLY)
        return task_rdd
        


## KITTI-360

In [ ]:
from psegs.datasets.kitti_360 import KITTI360SDTable
class KITTI360OurFusedClouds(KITTI360SDTable):
    INCLUDE_FISHEYES = False
    INCLUDE_FUSED_CLOUDS = False  # Use our own fused clouds

class KITTI360OurFusedWorldCloudTable(FusedLidarCloudTableBase):
    SRC_SD_TABLE = KITTI360OurFusedClouds
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        spark.catalog.dropTempView('culi_tasks_df')
        print('Building tasks table for %s ...' % segment_uri.segment_id)
        spark.sql("""
          CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
          SELECT 
              CONCAT(uri.segment_id, '.', uri.extra.`kitti-360.frame_id`) AS task_id,
              FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
              COLLECT_LIST(point_cloud) AS point_clouds
          FROM datums
          WHERE 
              uri.topic LIKE '%cuboid%' OR uri.topic LIKE '%lidar%'
          GROUP BY task_id
        """)
        tasks_df = spark.sql('SELECT * FROM culi_tasks_df')
        print('... done.')
        return tasks_df.rdd


## NuScenes

In [ ]:
# !pip3 install nuscenes-devkit==1.1.2
from psegs.datasets.nuscenes import NuscStampedDatumTableBase, NuscStampedDatumTableLabelsAllFrames
class NuscFusedWorldCloudTableBase(FusedLidarCloudTableBase):
    #SRC_SD_TABLE = NuscStampedDatumTableBase -- Subclass must pick a NuScenes configuration!
    
    SPLITS = ['train_detect', 'train_track']
    
    @classmethod
    def _filter_ego_vehicle(cls, cloud_ego):
        # Note: NuScenes authors have already corrected clouds for ego motion:
        # https://github.com/nutonomy/nuscenes-devkit/issues/481#issuecomment-716250423
        # But have not filtered out ego self-returns
        cloud_ego = cloud_ego[np.where(  ~(
                        (cloud_ego[:, 0] <= 1.5) & (cloud_ego[:, 0] >= -1.5) &  # Nusc lidar +x is +right
                        (cloud_ego[:, 1] <= 2.5) & (cloud_ego[:, 0] >= -2.5) &  # Nusc lidar +y is +forward
                        (cloud_ego[:, 1] <= 1.5) & (cloud_ego[:, 0] >= -1.5)    # Nusc lidar +z is +up
        ))]
        return cloud_ego
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        spark.catalog.dropTempView('culi_tasks_df')
        print('Building tasks table for %s ...' % segment_uri.segment_id)
        if cls.SRC_SD_TABLE.LABELS_KEYFRAMES_ONLY:
            # For Nusc: group by nuscenes-sample-token WITH KEYFRAMES
            spark.sql("""
              CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
              SELECT 
                  uri.extra.`nuscenes-sample-token` AS task_id,
                  FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
                  COLLECT_LIST(point_cloud) AS point_clouds
              FROM datums
              WHERE 
                uri.extra.`nuscenes-is-keyframe` = 'True' AND (
                  uri.extra['nuscenes-label-channel'] is NULL OR 
                  uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
                ) AND (
                  uri.topic LIKE '%cuboid%' OR
                  uri.topic LIKE '%lidar%'
                )
              GROUP BY task_id
            """)
        else:
            # For Nusc: group by nuscenes-sample-token WITH ALL FRAMES
            spark.sql("""
              CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
              SELECT 
                  CONCAT(uri.segment_id, '.', uri.timestamp) AS task_id,
                  FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
                  COLLECT_LIST(point_cloud) AS point_clouds
              FROM datums
              WHERE 
                (
                  uri.extra['nuscenes-label-channel'] is NULL OR 
                  uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
                ) AND (
                  uri.topic LIKE '%cuboid%' OR
                  uri.topic LIKE '%lidar%'
                )
              GROUP BY task_id
              HAVING SIZE(cuboids) > 0 AND SIZE(point_clouds) > 0
            """)
        
        tasks_df = spark.sql('SELECT * FROM culi_tasks_df')
        print('... done.')
        return tasks_df.rdd

class NuscFusedWorldCloudKeyframesOnlyTable(NuscFusedWorldCloudTableBase):
    SRC_SD_TABLE = NuscStampedDatumTableBase

class NuscFusedWorldCloudAllFramesTable(NuscFusedWorldCloudTableBase):
    SRC_SD_TABLE = NuscStampedDatumTableLabelsAllFrames
    

## Start Spark

In [ ]:
from psegs.spark import NBSpark
spark = NBSpark.getOrCreate()

## Build Fused Lidar Assets

```
docker --context default run -it --name=potree_viewer --rm --net=host -v `pwd`:/shared  jonazpiazu/potree
```

In [ ]:
T = KITTI360OurFusedWorldCloudTable
rdds = T._create_datum_rdds(spark)
print([r.count() for r in rdds])

seg_uris = T.get_all_segment_uris()
samp = T.get_sample(seg_uris[0], spark=spark)


In [ ]:
print([lc.sensor_name for lc in samp.lidar_clouds][:10])
c = samp.lidar_clouds[0]#[lc for lc in samp.lidar_clouds if lc.sensor_name == '11002'][0]
print(c.get_cloud().shape)
imshow(c.get_bev_debug_image(x_bounds_meters=None, y_bounds_meters=None))
imshow(c.get_front_rv_debug_image(y_bounds_meters=None, z_bounds_meters=None))

## Compute Candidate Optical Flow Pairs

## Render Optical Flow